In [1]:
import sys
sys.path.append('..')
import pandas as pd

In [2]:
from utils.utils import *

In [13]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [14]:
train_p = preprocessing(train.copy())
test_p = preprocessing(test.copy())

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\daniel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\daniel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
train_p.sample(5, random_state=7)

,id,keyword,location,text,target
6106,8718,sinking,NaN,theres chance get gander sinking ship tna cant...,0
5025,7166,mudslide,"Chiswick, London",great new recipes mudslide cake sorry stew gbbo,0
7022,10066,typhoon,USA,breaking news global precipitation measurement...,1
2331,3355,demolition,USA,epa begins demolition homes toxic area buffalo,1
3354,4800,evacuated,"Hensley Street, Portland",katunews remains closed brush fire burns acres,1


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_p.text.values,
                                                    train_p.target.values,
                                                    stratify=train_p.target.values,
                                                    test_size=0.2, random_state=1)

In [19]:
tfidf = TfidfVectorizer(min_df=2, max_df=0.5, ngram_range=(1, 2))
train_tfidf = tfidf.fit_transform(X_train)
test_tfidf = tfidf.transform(X_test)
test = tfidf.transform(test_p.text)

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score

In [21]:
clf = LogisticRegression(C=1.0)
scores = cross_val_score(clf, train_tfidf, y_train, cv=5, scoring="f1")
scores

array([0.74176408, 0.72391304, 0.73513514, 0.76756757, 0.72473118])

In [22]:
clf.fit(train_tfidf, y_train)

LogisticRegression()

In [23]:
f1_score(y_test, clf.predict(test_tfidf))

0.7517006802721089

In [24]:
clf.predict(test)

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [25]:
submission = pd.DataFrame({'id': pd.read_csv('../data/test.csv').id.values,
                           'target': clf.predict(test)})

In [27]:
submission.to_csv('../out/LogRegre.csv', index=False)